In [ ]:
from DistMLIP.implementations.uma.ase_calculator import FAIRChemCalculator_Dist
from fairchem.core import FAIRChemCalculator

from pymatgen.core import Structure, Lattice
from pymatgen.io.ase import AseAtomsAdaptor

import os

In [ ]:
# Insert your atoms
struct = Structure.from_spacegroup("Pm-3m", Lattice.cubic(3.5), ["Li", "Mn"], [[0, 0, 0], [0.5, 0.5, 0.5]])
struct.perturb(3)
struct.make_supercell((30, 30, 30))
print(f"There are {len(struct)} atoms.")
atoms = AseAtomsAdaptor().get_atoms(struct)

In [ ]:
from fairchem.core import pretrained_mlip

# Load the UMA model on cpu and update the device
devices = [6, 7]
predictor = pretrained_mlip.get_predict_unit("uma-s-1", device="cpu")

# Set the device, this device should be the 0th device in the devices list
predictor.device = f"cuda:{devices[0]}"

# Create the FAIRChemCalculator, then make a distributed version
dist_calc = FAIRChemCalculator(predictor, task_name="omol")
dist_calc = FAIRChemCalculator_Dist.from_existing(dist_calc)

# Enable distributed mode and specify the GPUs to use
dist_calc.enable_distributed_mode(devices)

In [ ]:
# You can set the number of threads to use via environment variable
os.environ["DISTMLIP_NUM_THREADS"] = "128"

# Performing a static calculation on the ASE atoms object
dist_calc.calculate(atoms, properties=dist_calc.implemented_properties, system_changes=[])

# Retrieiving results
dist_calc.results['energy'], dist_calc.results['forces'], dist_calc.results['stress']

In [ ]:
# You can use FAIRChemCalculator_Dist the same way as FAIRChemCalculator. An example running MD simulation using ASE:
from ase import units
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet

MaxwellBoltzmannDistribution(atoms, temperature_K=300)
atoms.calc = dist_calc

dyn = VelocityVerlet(atoms, 0.5 * units.fs)

dyn.run(10)